### Belief Propagation + Ordered Statistics Decoding 

In [2]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple
import galois
import sympy
import qiskit
import pennylane as qml

In [3]:
def tanner_to_parity_check_mat(G: nx.Graph) -> np.array:
    """
    Converts Tanner graph 'G' into parity check matrix

    Parameters:
    -----------
    G - Tanner graph
    """
    num_var_nodes = 0
    num_check_nodes = 0
    check_node_list = []
    nodes_dict = {}
    G_nodes = list(G.nodes)
    G_edges = list(G.edges)
    for node in G_nodes:
        if (node[0] == 'v'):
            nodes_dict[node] = num_var_nodes
            num_var_nodes += 1
        else:
            nodes_dict[node] = num_check_nodes
            num_check_nodes += 1
    
    mat = np.zeros((num_check_nodes, num_var_nodes))
    for edge in G_edges:
        if (edge[0][0] == 'c'):
            mat[nodes_dict[edge[0]]][nodes_dict[edge[1]]] = 1
        else:
            mat[nodes_dict[edge[1]]][nodes_dict[edge[0]]] = 1
            
    return mat

def parity_check_mat_to_tanner(mat: np.array) -> nx.Graph:
    """ 
    Converts parity check matrix 'mat' into Tanner graph

    Parameters:
    -----------
    mat - parity check matrix
    """
    num_var_nodes = mat.shape[1]
    num_check_nodes = mat.shape[0]
    tan_graph = nx.Graph();
    for i in range(num_var_nodes):
        tan_graph.add_node("v" + str(i))

    for j in range(num_check_nodes):
        tan_graph.add_node("c" + str(j))

    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            if (mat[i][j] == 1):
                tan_graph.add_edge("c" + str(i), "v" + str(j))
    
    return tan_graph

In [4]:
def belief_prop(H: np.array, s: np.array, p: float, max_iter: int) -> Tuple:
    """ 
    Belief Propagation Algorithm for Decoding LDPC Codes

    Parameters:
    -----------
    H - parity-check matrix corresponding to either X or Z checks
    s - Error syndrome
    p - Channel error rate for chosen noise channel
    max_iter - Maximum number of iterations to run BP algorithm for
    """
    data_to_parity = np.zeros((len(H[0]),len(H)), dtype=float)
    parity_to_data = np.zeros((len(H), len(H[0])), dtype=float)
    H_tanner_graph = parity_check_mat_to_tanner(H)
    
    # Channel Log Likelihood Ratio
    p_l = np.log((1 - p)/p)
    P_1 = np.zeros((len(H[0]),), dtype=float)
    e_BP = np.zeros((len(H[0]),), dtype=float)

    # (1) Initialization
    for edge in H_tanner_graph.edges:
        data_node_num = int(edge[0][1:])
        parity_node_num = int(edge[1][1:])
        data_to_parity[data_node_num][parity_node_num] = p_l 

    for iter in range(1, max_iter + 1):
        # Scaling Factor
        a = 1 - 2**(-1 * iter)

        # (2) Parity to Data Messages
        for edge in H_tanner_graph.edges:
            parity_node_num = int(edge[1][1:])
            data_node_num = int(edge[0][1:])

            # Get list of neighbors of current parity_node set minus the current data node
            V = list(nx.neighbors(H_tanner_graph, edge[1]))
            V.remove(edge[0])

            # Get messages from elements of V to current parity node
            data_to_par_msgs = [data_to_parity[int(v[1:])][parity_node_num] for v in V]
            w = np.min([np.abs(msg) for msg in data_to_par_msgs])
            parity_to_data[parity_node_num][data_node_num] = ((-1) ** s[parity_node_num]) * a * np.prod(np.sign(data_to_par_msgs)) * w 

        # (3) Data to Parity Messages
        for edge in H_tanner_graph.edges:
            data_node_num = int(edge[0][1:])
            parity_node_num = int(edge[1][1:])

            # Get list of neighbors of current data node set minus the current parity node
            U = list(nx.neighbors(H_tanner_graph, edge[0]))
            U.remove(edge[1])

            # Get messages from elements of U to current data node
            par_to_data_msgs = [parity_to_data[int(u[1:])][data_node_num] for u in U]
            data_to_parity[data_node_num][parity_node_num] = p_l + np.sum(par_to_data_msgs)

        # Hard Decision 
        for data_node in [node for node in H_tanner_graph.nodes if node[0] == 'v']:
            data_node_num = int(data_node[1:])

            # Get list of neighbors of current data node 
            U = list(nx.neighbors(H_tanner_graph, data_node))

            # Get messages from elements of U to current data node
            par_to_data_msgs = [parity_to_data[int(u[1:])][data_node_num] for u in U]
            P_1[data_node_num] = p_l + np.sum(par_to_data_msgs)
            e_BP[data_node_num] = -1 * np.sign(P_1[data_node_num])
        
        
        # (4) Termination Check
        if (np.array_equal(np.dot(H, e_BP), s)):
            return True, e_BP, P_1 

    return False, e_BP, P_1

# Turn to Ordered Statistics Decoding if BP fails to converge
def OSD_0(H: np.array, P_1: np.array, s: np.array) -> np.array:
    """ 
    The Ordered Statistics Decoding (OSD) Zero algorithm is a post-processing 
    algorithm utilized when BP fails to converge 

    Parameters:
    -----------
    H - parity check matrix
    P_1 - BP soft decision vector
    s - Error syndrome 

    Returns:
    --------
    Error string
    """
    S = np.array()

    # Get the rank of the parity check matrix
    H_rank = rank(matrix(H))

    # Maintain a mapping between elements of the BP soft-decision vector and bit positions 
    P_1_dict = {p:i for p in P_1 for i in range(len(P_1))}
    P_1_sorted = np.sort(P_1)
    P_1_sorted_pos = [P_1_dict[p] for p in P_1_sorted]
    
    # Rearrange columns of H to match the reordered soft-decision vector
    idx = np.empty_like(P_1_sorted_pos)
    H[:] = H[:, idx]

    # Select first RANK(H) linearly independent columns of above rearrangement
    _, inds = sympy.Matrix(H).rref()
    H_S = np.vstack((H[:][inds[i]] for i in range(0, H_rank))).T

    # Calculate the OSD-0 solution on the basis-bits
    e_S = np.linalg.inv(H_S) * s
    e_ST = np.hstack((e_S, np.zeros((len(H[0]) - H_rank,))))

    # Map the OSD-0 solution to the original bit-ordering
    e_OSD = np.zeros((len(H[0]),))
    for i in range(len(P_1_sorted_pos)):
        e_OSD[P_1_sorted_pos[i]] = e_ST[i]

    return e_OSD

def OSD_0_Plus(H: np.array, P_1: np.array, s: np.array, e_T: np.array) -> np.array:
    """ 
    The Higher Order OSD algorithm is a post-processing 
    algorithm utilized when BP fails to converge, building 
    on OSD-0 Algorithm

    Parameters:
    -----------
    H - parity check matrix
    P_1 - BP soft decision vector
    s - Error syndrome 
    e_T - Choice of error bits on remaining bits that aren't getting flipped with high enough probability

    Returns:
    --------
    Error string 
    """
    GF = galois.GF(2)
    S = np.array()

    # Get the rank of the parity check matrix
    H_rank = rank(matrix(H))

    # Maintain a mapping between elements of the BP soft-decision vector and bit positions 
    P_1_dict = {p:i for p in P_1 for i in range(len(P_1))}
    P_1_sorted = np.sort(P_1)
    P_1_sorted_pos = [P_1_dict[p] for p in P_1_sorted]
    
    # Rearrange columns of H to match the reordered soft-decision vector
    idx = np.empty_like(P_1_sorted_pos)
    H[:] = H[:, idx]

    # Select first RANK(H) linearly independent columns of above rearrangement
    _, inds = sympy.Matrix(H).rref()
    H_S = np.vstack((H[:][inds[i]] for i in range(0, H_rank))).T
    H_T = np.vstack(H[:][inds[i]] for i in range (H_rank, len(H[0])))

    # Calculate the OSD-0 solution on the basis-bits
    e_S = np.linalg.inv(H_S) * s
    e_ST_1 = GF(np.linalg.inv(H_S) * e_S + np.linalg.inv(H_S) * H_T * e_T)
    e_ST_2 = GF(e_T)
    e_ST = np.hstack((e_ST_1, e_ST_2))

    # Map the OSD-0 solution to the original bit-ordering
    e_OSD = np.zeros((len(H[0]),))
    for i in range(len(P_1_sorted_pos)):
        e_OSD[P_1_sorted_pos[i]] = e_ST[i]

    return e_OSD

In [20]:
def gen_rand_pauli_error(num_qubits: int, r:float) -> Tuple:
    """
    Generate a random pauli error on 'num_qubits' qubits
    """
    error_op_X = [None for i in range(num_qubits)]
    error_op_Z = [None for i in range(num_qubits)] 
    for i in range(num_qubits):
        z = np.random.uniform()
        if (z >= 0 and z <= r/3):
            error_op_X[i], error_op_Z[i] = 1, 0
        elif (z > r/3 and z <= (2*r)/3):
            error_op_X[i], error_op_Z[i] = 1, 1
        elif (z > (2 * r)/3 and z <= r):
            error_op_X[i], error_op_Z[i] = 0, 1
        else:
            error_op_X[i], error_op_Z[i] = 0, 0
    return error_op_X, error_op_Z

    

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1]
